In [3]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

import numpy as np
import pandas as pd
import os, sys
import requests

sys.path.insert(0, '..')

from transformers.data_container import *
from transformers.transformers import TimeSeriesTransformer, MeanSeriesTransformer, TimeSeriesWindowTransformer

/Users/iwitaly/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Usage example will be shown based on open source time series [data set](http://timeseriesclassification.com/Downloads/FordA.zip).

First think one need to do is read data. We use `urlopen` function from built in urllib Python package to download data set in memory. We also shring the length of each series.

In [10]:
url = "http://timeseriesclassification.com/Downloads/FordA.zip"
series_offset = 505

In [11]:
url = urlopen(url)
zipfile = ZipFile(BytesIO(url.read()))
lines = zipfile.open('FordA/FordA.csv').readlines()
lines = [l.decode('utf-8') for l in lines]
lines = lines[series_offset:]

**lines** now is a list of strings with of timeseries in comma separeted format

505 is a offset for the beginning of seriases

In [12]:
lines = [list(map(float, l.split(','))) for l in lines]

In [13]:
lines[0][:10]

[1.1871,
 0.4096,
 -0.43154,
 -1.231,
 -1.9055,
 -2.3824,
 -2.588,
 -2.5018,
 -2.1353,
 -1.574]

Now **lines** is a list of list of floats. Let's convert each embedded list into more convince pandas.Series object.

In [14]:
lines = [pd.Series(l) for l in lines]

In [15]:
lines[0][:10]

0    1.18710
1    0.40960
2   -0.43154
3   -1.23100
4   -1.90550
5   -2.38240
6   -2.58800
7   -2.50180
8   -2.13530
9   -1.57400
dtype: float64

Now we have a list of pandas.Series objects. Next thing we do is encapsulate list into another series called MultiSeries. Thus list of lists of float became into MultiSeries of pandas.Series objects.

We have a global indes for MuiltiSeries and each pandas.Series has it's own index.

In [16]:
X = MultiSeries(lines)

In [17]:
X.head()

0    0      1.187100
1      0.409600
2     -0.43154...
1    0      0.094261
1      0.310310
2      0.53060...
2    0     -1.157000
1     -1.592600
2     -1.50960...
3    0      0.356960
1      0.300850
2      0.24314...
4    0      0.307980
1      0.370350
2      0.26015...
dtype: object
data_type: <class 'pandas.core.series.Series'>

Output might seems a bit messy. It prints MultiSeries of pandas.Series and **data_type** property. This property shown the type of underlying data into MultiSeries.

**X** now is a MultiSeries of pd.Serieses. It means, that every element of this MultiSeries is pd.Series.

**X** supports all methods as general pd.Series does.

## Transformers

One of the common task in data science that was a motivation for this project is to extract features from some complex objects (for example series) and than do a fancy machine learning.

Having a MultiSeries of pandas.Series one would like to extract features from each Series. That's where Transformers take place. Let's try on example.

The first simple example of transformer is *TimeSeriesWindowTransformer*. This transformer calculates moving average with given window size.

In [19]:
tr = TimeSeriesWindowTransformer(windows_size=5)
tr.fit()
transformed_series = tr.transform(X)

In [20]:
transformed_series.head()

0    0           NaN
1           NaN
2           Na...
1    0           NaN
1           NaN
2           Na...
2    0           NaN
1           NaN
2           Na...
3    0           NaN
1           NaN
2           Na...
4    0           NaN
1           NaN
2           Na...
dtype: object
data_type: <class 'pandas.core.series.Series'>

Of course, with a windows_size = 5 first 4 elements are NaN.

In [21]:
transformed_series[0].head(10)

0         NaN
1         NaN
2         NaN
3         NaN
4   -0.394268
5   -1.108168
6   -1.707688
7   -2.121740
8   -2.302600
9   -2.236300
dtype: float64

Let's try another transformer, probably the most common. It extract several quantitative features from each pandas.Series like mean, std, quantiles. You can pass you own list of features. As a result we have an object **MultiDataFrame**.

In [22]:
tr = TimeSeriesTransformer()
tr.fit()
transformed_series = tr.transform(X)

In [23]:
type(transformed_series)

transformers.data_container.data_container.MultiDataFrame

In [25]:
transformed_series.head()

,max,mean,median,min,quantile_25,quantile_75,quantile_90,quantile_95,std
0,2.5263,0.001995,0.011186,-2.7875,-0.73635,0.74192,1.2534,1.5463,0.999998
1,2.6291,0.001997,-0.024726,-2.4357,-0.67411,0.65808,1.3478,1.6595,0.999997
2,2.6072,-0.001996,0.060685,-3.0132,-0.67588,0.70123,1.2591,1.5184,1.000000
3,2.6431,-0.001997,-0.022668,-2.7275,-0.66265,0.56858,1.4102,1.8094,0.999997
4,3.2398,-0.001995,-0.048518,-3.0085,-0.70775,0.64898,1.2540,1.6699,1.000001


**MultiDataFrame** is an abstract container based on pandas.DataFrame that can store **MultiSeries** objects.

The main feature of **MultiDataFrame** is that you have columns of **MultiSeries** of any **data_type** with ANY objects. For example, one may has a data set Serieses, Images, Texts, plain Numbers, any custom objects. Imagine a 2d data container that stores it. That one would like to write a chain of transformers to create a simple 2d matrix with number ready-to-sklearn predictor.

Let's see an example with that. First, let's create a **MultiDataFrame**.

Let's **Y** will be a labels for each row.

In [44]:
Y = np.random.binomial(1, 0.5, X.shape[0])
Y = MultiSeries(Y)

In [46]:
df = MultiDataFrame({
    'series': X,
    'labels': Y
})

In [48]:
df.head()

,labels,series
0,1,0 1.187100 1 0.409600 2 -0.43154...
1,1,0 0.094261 1 0.310310 2 0.53060...
2,1,0 -1.157000 1 -1.592600 2 -1.50960...
3,0,0 0.356960 1 0.300850 2 0.24314...
4,1,0 0.307980 1 0.370350 2 0.26015...


In [49]:
from transformers.transformers import PipeLineChain

In [56]:
chain = PipeLineChain([
    ('moving average trans', TimeSeriesWindowTransformer(window=3)),
    ('extract features', TimeSeriesTransformer())
])
chain.fit(df)

In [59]:
transformed_df = chain.transform(df)

In [60]:
transformed_df.head()

,labels,series_max,series_mean,series_median,series_min,series_quantile_25,series_quantile_75,series_quantile_90,series_quantile_95,series_std
0,1,2.401300,-0.000130,0.008040,-2.683500,-0.707967,0.705327,1.231112,1.509677,0.962310
1,1,2.548033,0.000039,-0.035743,-2.383933,-0.658162,0.632137,1.295828,1.655187,0.974004
2,1,2.510400,0.002398,0.057712,-2.809300,-0.642350,0.672370,1.158198,1.449777,0.946934
3,0,2.573633,-0.000626,-0.016575,-2.647367,-0.641810,0.549031,1.390788,1.762080,0.979036
4,1,3.024933,-0.001421,-0.041690,-2.817867,-0.668768,0.634767,1.205933,1.560040,0.949124


Now this data set is ready for a plain sklearn estimators!